# Working with containers

Here I will show some of the operations associated with docker containers.

### `docker ps` - show containers

In [4]:
%%bash
docker run -itd --rm --name test_container python:3.10 &> /dev/null
docker ps
docker stop test_container &> /dev/null

CONTAINER ID   IMAGE         COMMAND     CREATED        STATUS                  PORTS     NAMES
b17984f2b882   python:3.10   "python3"   1 second ago   Up Less than a second             test_container


##### `-a` show all containers

By default, inactive containers are hidden.

In [8]:
%%bash
docker run --name temp_container ubuntu &> /dev/null
echo "========no option -a=============="
docker ps
echo "=========with option -a============="
docker ps -a
docker rm temp_container &> /dev/null

========no option -a==============
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
=========with option -a=============
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                              PORTS     NAMES
ad0439c23847   ubuntu    "/bin/bash"   Less than a second ago   Exited (0) Less than a second ago             temp_container


### `docker history` - show layers of the package

### `docker run` - create the container from image

In [9]:
%%bash
echo "================before start====================="
docker ps -a
echo "===============after start======================"
docker run --name temp_container ubuntu &> /dev/null
docker ps -a
docker rm temp_container

================before start=====================
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
===============after start======================
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
cd0f144063c5   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             temp_container
temp_container


##### `-rm` - automatic container removal

After stopping, the container is removed. In the following example there are two containers of ubuntu, one with the `--rm` option and the other without. Both containers stops by default - but only one stays with `STATUS == Exited`.

In [12]:
%%bash
docker run --name no_rm_ubuntu ubuntu
docker run --rm --name ubuntu_with_rm ubuntu
docker ps -a
docker rm no_rm_ubuntu &> /dev/null

CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
7acc3cc0c61c   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             no_rm_ubuntu


##### `--name` - set name for the container

In [2]:
%%bash
docker run --rm --name unbuntu_container -d ubuntu
docker ps
docker stop unbuntu_container

7116dcdaa0a131a8507f9cbe149719498456fe356a23f1ddfc48f65080cfe660
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                  PORTS     NAMES
7116dcdaa0a1   ubuntu    "/bin/bash"   1 second ago   Up Less than a second             unbuntu_container
unbuntu_container


##### `-d` - make container not to lock CLI

In [15]:
%%bash
docker run --rm -d --name temp_nginx nginx &> /dev/null
docker ps
docker stop temp_nginx &> /dev/null

CONTAINER ID   IMAGE     COMMAND                  CREATED                  STATUS                  PORTS     NAMES
09175cb8c6d9   nginx     "/docker-entrypoint.…"   Less than a second ago   Up Less than a second   80/tcp    temp_nginx


##### `-i` - interactive mode

Mode that allows to interapt with container.

Unfortunately it is not possible to show how this mode will work in jupyter. So here is a gif of me experimenting with the following command:

`docker run -i --rm --name test_ubuntu ubuntu`

<img src="containers_files/docker -i option.gif"></img>

In the example, the ubuntu container has been started and is stopped by default. The `-i` option causes it to lock the terminal. Extremely interesting that you will be able to send messages to the container and get results (as I showed with `ls` and `exit` commands), but you won't get the description of the user common for Linux terminal.

##### `-t` turn on TTY

TTY is something on backend language. The point here is that in combination with the `-i` option you can make will show you the "description" of the user which was mentioned in the previous section. And the terminal will be blocked waiting for input. But this option don't allow you to send anything to the container.

The following `.gif` show what you will get in case using command `docker run -t --rm --name temp_ubuntu ubuntu`.

<img src="containers_files/docker -t option.gif"></img>

##### `-it` - combination

Both options were discovered above, but they are really often used in combination. So Ubuntu provides full CLI interface with this options.

<img src="containers_files/docker -it option.png"></img>

##### `-e` - environment variables

In [1]:
%%bash
docker run --rm -itd --name e_example -e TEST_VAR="value" ubuntu &> /dev/null
docker exec e_example bash -c "echo \$TEST_VAR"
docker stop e_example &> /dev/null

value


You can also pass the values of the variables of the system you are calling from.

In [3]:
%%bash
docker run --rm -itd --name e_example -e FEDOR_TEST_VAR=$HOME ubuntu &> /dev/null
docker exec e_example bash -c "echo \$FEDOR_TEST_VAR"
docker stop e_example &> /dev/null

/home/fedor


### `docker start` - activate exited container

In the next cell, I build a ubuntu container that prints "hello world" when it starts. Like all standard ubuntu containers it quits, which we can see in the `STATUS` of the `ps` command. Then by `docker start` the container is reactivated.

For some reason it only works when I use the `-i` option for `docker start`.

In [16]:
%%bash
cd containers_files
docker build -t test_ubuntu . &> /dev/null
echo "=====First start====="
docker run --name u1 test_ubuntu
echo "=====List of all containers====="
docker ps -a
echo "=====Restart====="
docker start -i u1
docker rm u1 &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====First start=====
hello world
=====List of all containers=====
CONTAINER ID   IMAGE         COMMAND                CREATED        STATUS                              PORTS     NAMES
af533b577c94   test_ubuntu   "echo 'hello world'"   1 second ago   Exited (0) Less than a second ago             u1
=====Restart=====
hello world


### `docker stop` - stop some container

In the following example, I start a container with Ubuntu in such a way that it stays running, which was proven by using `docker ps`. But after using `docker stop` the same container is in the state "Exited".

In [20]:
%%bash
docker run -itd --name test_ubuntu ubuntu &> /dev/null
echo "=====Just started container====="
docker ps -a
docker stop test_ubuntu &> /dev/null
echo "=====Stoped container (STATUS \"Exited\")====="
docker ps -a
docker rm test_ubuntu &> /dev/null

=====Just started container=====
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                  PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   Less than a second ago   Up Less than a second             test_ubuntu
=====Stoped container (STATUS "Exited")=====
CONTAINER ID   IMAGE     COMMAND       CREATED          STATUS                                PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   10 seconds ago   Exited (137) Less than a second ago             test_ubuntu


### `docker exec` - run a command in the container

In the next command I start ubuntu conteiner. Then using `docker exec' I run the `ls' command in the container and get a typical Linux result for that command.

In [24]:
%%bash
docker run -itd --rm --name test_ubuntu ubuntu &> /dev/null
docker exec test_ubuntu ls
docker stop test_ubuntu &> /dev/null

bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var


##### `-it` - take control of the container

It looks like the options individually have the same properties as in the `docker run` command. But the important thing is that you can take control of the detached container with this command.<br> 
The following example shows how I can get control in detached ubuntu by running the `bash` command with named options.

<img src="containers_files/docker exec -it.gif"></img>